In [3]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
img1 = cv.imread('/examples/img1.ppm')
img2 = cv.imread('/examples/img2.ppm')
gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)
bf = cv.BFMatcher(cv.NORM_L2)
matches = bf.knnMatch(des1, des2, k=2)
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)
MIN_MATCH_COUNT = 10
if len(good_matches) >= MIN_MATCH_COUNT:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
    warped_img2 = cv.warpPerspective(img2, H, (img1.shape[1] + img2.shape[1], img1.shape[0]))
    warped_img2[:img1.shape[0], :img1.shape[1]] = img1
    cv.imshow('Stitched Image result', warped_img2)
    cv.waitKey(0)
    cv.destroyAllWindows()

else:
    print("Not enough matches for the image - %d/%d" % (len(good_matches), MIN_MATCH_COUNT))